In [15]:
import polars as pl

In [16]:
df_preppet = pl.read_parquet('kjoretoyinfo_preppet.parquet')

In [17]:
df_preppet

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,er_elektrisk,tekn_farge,tekn_kjtgrp
date,date,i64,i64,str,str,date,bool,str,i64
2021-10-13,2021-10-13,2,3900,"""VITO""","""2""",2025-10-13,false,"""05""",315
2020-01-29,2020-01-29,2,5750,"""CADDY""","""2""",2026-01-29,false,"""06""",315
2023-11-29,2023-11-29,1,2600,"""Mustang Mach-E…","""5""",2027-11-29,true,"""06""",101
2020-01-27,2020-01-27,2,5750,"""CADDY""","""2""",2024-01-27,false,"""01""",315
2019-01-30,2019-01-30,2,2600,"""F-150 Platinum…","""2""",2024-05-03,false,"""06""",315
…,…,…,…,…,…,…,…,…,…
2003-11-27,2017-06-10,0,4350,"""ZAFIRA-A""","""1""",2021-04-30,false,"""11""",101
2014-01-17,2018-07-31,2,2600,"""RANGER""","""2""",2024-06-30,false,"""05""",315
2008-03-31,2023-07-28,0,3930,"""MX-5""","""1""",2025-08-16,false,"""05""",101


In [18]:
førstegangsregistreres_biler = df_preppet.filter(pl.col("tekn_reg_f_g_n").is_not_null())

In [19]:
flest_førstegangsregistering = (
    førstegangsregistreres_biler.group_by(førstegangsregistreres_biler['tekn_reg_f_g_n'].dt.month())
    .agg(pl.col("tekn_reg_f_g_n").count().alias("count"))
    .sort("count", descending=True)
    .head(1)
)
print(f"Måneden med flest førstegangsregistreringer er: {flest_førstegangsregistering}")

Måneden med flest førstegangsregistreringer er: shape: (1, 2)
┌────────────────┬────────┐
│ tekn_reg_f_g_n ┆ count  │
│ ---            ┆ ---    │
│ i8             ┆ u32    │
╞════════════════╪════════╡
│ 6              ┆ 248455 │
└────────────────┴────────┘


In [27]:
# Les Parquet-filen inn i en Polars DataFrame
df = pl.read_parquet("kjoretoyinfo_preppet.parquet")

s = pl.Series("måned", ["tekn_reg_f_g_n"])
s.value_counts() 
grouped_df = df.group_by("Måned").agg(pl.sum("Antall_registreringer"))


max_month = grouped_df.sort("sum_Antall_registreringer", descending=True).select("Måned").first()

print(f"Måneden med flest førstegangsregistreringer i Norge er: {max_month['Måned']}")

ColumnNotFoundError: Måned

Error originated just after this operation:
DF ["tekn_reg_f_g_n", "tekn_reg_eier_dato", "tekn_aksler_drift", "tekn_merke"]; PROJECT */10 COLUMNS; SELECTION: "None"